In [1]:
    # import libraries
import glob                               #can be used to find the target files
import matplotlib.pyplot as plt
import nibabel as nib                   # to read and write neuron imaging data 
import numpy as np                        # for matrix calculating
import subprocess
from PIL import Image 
import os,shutil
from scipy import signal
from skimage import color, img_as_ubyte, io


In [2]:
file_script = 'results_script1'
if not os.path.exists(file_script):
    os.mkdir(file_script)

In [3]:
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/BVECTOR.scheme','results_script1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/fsltensor_FA.nii.gz','results_script1')
shutil.copy('/home/zhiwei/Desktop/semester-projects/original_file/3000/fsltensor_V3.nii.gz','results_script1')

CMD = 'shutil.copy(“fsltensor_V1.nii.gz”,”results_script1”)'
subprocess.call(CMD,shell=True)


In [4]:
cd results_script1

/home/zhiwei/Desktop/semester-projects/results_script1


## generate original tractography and get fib_original, hdr_original

In [5]:
data = nib.load('fsltensor_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask.nii')

In [6]:
CMD='tckgen fsltensor_V3.nii.gz origin0.015.tck -algorithm FACT -seed_image mask.nii -mask mask.nii -select 3000 -step 0.015'
subprocess.call(CMD, shell = True)

0

In [7]:
CMD = 'TractConverter.py -i origin0.015.tck -o origin0.015.trk -a fsltensor_FA.nii.gz'
subprocess.call(CMD,shell=True)

0

In [8]:
fib_original,hdr_original=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/origin0.015.trk')

## generate synthetic CLARITY data

In [9]:
CMD='tckmap origin0.015.tck syn_CLARITY.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0


## calculate structure tensor

In [10]:
def doggen(sigma=None, X=None, Y=None, Z=None):
    halfsize = np.ceil(3 * np.max(sigma))       #3*the maximum number of sequence sigma, ceil: shangxian
    x = np.arange(-halfsize,halfsize+1)         # sequence from -halfsize to halfsize+1

    if len(sigma) == 1:
        if X is None:
            X = x
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) )
    if len(sigma) == 2:
        if X is None or Y is None:
            [X,Y] =np.meshgrid(x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) )
    if len(sigma) == 3:
        if X is None or Y is None or Z is None:
            [X,Y,Z] =np.meshgrid(x,x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) ) * np.exp( (-1)*np.power(Z,2)/(2 * np.power(sigma[2],2)))
    if len(sigma) > 3:
        print ('Only support up to dimension 3')
    
    return k /np.sum(np.abs(k))

def gradCompute(img, dogsigma):
    dogkercc = doggen([dogsigma, dogsigma, dogsigma])
    dogkerrr = np.transpose( dogkercc, (1, 0, 2) )
    dogkerzz = np.transpose( dogkercc, (0, 2, 1) )

    gcc = signal.convolve(img, dogkercc, mode = 'valid')
    grr = signal.convolve(img, dogkerrr, mode = 'valid')
    gzz = signal.convolve(img, dogkerzz, mode = 'valid')

    # Gradient products
    gp = type('', (), {})()
    gp.gprrrr = grr * grr
    gp.gprrcc = grr * gcc
    gp.gprrzz = grr * gzz
    gp.gpcccc = gcc * gcc
    gp.gpcczz = gcc * gzz
    gp.gpzzzz = gzz * gzz

    # Gradient amplitude
    ga = np.sqrt(gp.gprrrr + gp.gpcccc + gp.gpzzzz)

    return ga, gp



## the optimal patch_size to retrieve the exact size of CLARITY as the originally synthetic one!! do not change this anymore!!

In [11]:
patch_size = 6.6

In [12]:

img_stack = nib.load('syn_CLARITY.nii.gz')
imgstack = img_stack.get_fdata()/255.0


# select sigma
dogsigma = [0.6]
vox_size = [15, 15, 15] #um
#define the voxel dimension to apply tensor
voxel_dim = np.array([patch_size,patch_size,patch_size]) # x,y,z in voxel dimension
volume_dim = (np.array(imgstack.shape)/voxel_dim).astype(np.int) # in voxel dimesion: removed voxel at the border

fsl_tensor = np.zeros((volume_dim[0], volume_dim[1], volume_dim[2], 6))
for x in range (0,volume_dim[0]):
    for y in range (0,volume_dim[1]):
        for z in range (0, volume_dim[2]):
            [ga,gp] = gradCompute(imgstack[int(x*voxel_dim[0]):int((x+1)*voxel_dim[0]), int(y*voxel_dim[1]):int((y+1)*voxel_dim[1]), int(z*voxel_dim[2]):int((z+1)*voxel_dim[2])], dogsigma)

            lowthreh = np.percentile(ga.reshape(-1), 20)
            highthreh = np.percentile(ga.reshape(-1), 80)

            mask = np.ones(ga.shape)
            mask[ga  < lowthreh] = 0
            mask[ga  > highthreh] = 0

            gprrrr = np.sum(gp.gprrrr[mask==1])
            gprrcc = np.sum(gp.gprrcc[mask==1])
            gprrzz = np.sum(gp.gprrzz[mask==1])
            gpcccc = np.sum(gp.gpcccc[mask==1])
            gpcczz = np.sum(gp.gpcczz[mask==1])
            gpzzzz = np.sum(gp.gpzzzz[mask==1])

            fsl_tensor[x,y,z] = np.array([[gprrrr, gprrcc, gprrzz,gpcccc, gpcczz, gpzzzz]])
affine = np.eye(4)
affine[0,0]=vox_size[0]*voxel_dim[0]*0.001
affine[1,1]=vox_size[1]*voxel_dim[1]*0.001
affine[2,2]=vox_size[2]*voxel_dim[2]*0.001
img_fsl_tensor = nib.Nifti1Image(fsl_tensor, affine)
nib.save(img_fsl_tensor, 'fsltensor_ftrial.nii.gz')
CMD='fsl5.0-fslmaths fsltensor_ftrial.nii.gz -tensor_decomp fsltensor_ftrial'
subprocess.call(CMD, shell=True)

/home/zhiwei/.local/lib/python2.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


0

## generate the mask for subsequent tractography generation

In [13]:
data = nib.load('fsltensor_ftrial_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask_syn.nii')

##  generate the tractography and subsequent CLARITY based on the synthetic CLARITY images

In [14]:
number = 100000

In [15]:
CMD='tckgen fsltensor_ftrial_V3.nii.gz before0.015.tck -algorithm FACT -seed_image mask_syn.nii -mask mask_syn.nii -select 100000 -step 0.015'
subprocess.call(CMD, shell = True)

0

In [16]:
CMD = 'TractConverter.py -i before0.015.tck -o before0.015.trk -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [17]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/before0.015.trk')

In [18]:
len(fib_before)

100000

In [19]:
for items in range(len(fib_original)):
    fib_before.append(fib_original[items])
nib.trackvis.write('fibers_before_all.trk',fib_before,hdr_before)

In [20]:
CMD = 'TractConverter.py -i fibers_before_all.trk -o fibers_before_all.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [21]:
CMD='tckmap fibers_before_all.tck syn_CLARITY_before.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0

## function for comparison of two tractography on a voxel base

In [22]:
def compClarity(volume1,volume2):
    error = 0.0
    for x in range(volume1.shape[0]):
        for y in range(volume1.shape[1]):
            for z in range(volume1.shape[2]):
                error = error + np.abs(volume1[x,y,z]-volume2[x,y,z])
    return error

## apply COMMIT to the generated tractography

In [23]:
path_commit = '/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT'
path = '/home/zhiwei/Desktop/semester-projects/results_script1'

In [24]:
trk_file_before = '/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before_all.trk'

In [25]:
from commit import trk2dictionary
trk2dictionary.run(filename_trk   = trk_file_before ,path_out= path_commit )
import commit
commit.core.setup()


-> Creating the dictionary from tractogram:
	* Segment position = COMPUTE INTERSECTIONS
	* Fiber shift X    = 0.000 (voxel-size units)
	* Fiber shift Y    = 0.000 (voxel-size units)
	* Fiber shift Z    = 0.000 (voxel-size units)
	* Points to skip   = 0
	* Min segment len  = 1.00e-03
	* Do not blur fibers
	* Loading data:
		* tractogram
			- 28 x 26 x 5
			- 0.0990 x 0.0990 x 0.0990
			- 103000 fibers
		* no mask specified to filter IC compartments
		* no dataset specified for EC compartments
		* output written to "/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT"
	* Generate tractogram matching the dictionary: 
	  [ 102751 fibers kept ]
   [ 12.0 seconds ]


In [26]:
mit=commit.Evaluation('.','results_script1')

In [27]:
mit.load_data('/home/zhiwei/Desktop/semester-projects/results_script1/syn_CLARITY.nii.gz','/home/zhiwei/Desktop/semester-projects/results_script1/BVECTOR.scheme')
mit.set_model('VolumeFractions')
hasISO=False
mit.model.set(hasISO)
mit.generate_kernels(regenerate=True)


-> Loading data:
	* DWI signal...
		- dim    = 191 x 178 x 39 x 1
		- pixdim = 0.015 x 0.015 x 0.015
	* Acquisition scheme...
		- 1 samples, 1 shells
		- 0 @ b=0 , 1 @ b=1000.0
   [ 0.0 seconds ]

-> Preprocessing:
	* There are no b0 volume(s) for normalization... [ min=0.00,  mean=0.26, max=30.00 ]
   [ 0.0 seconds ]

-> Simulating with "Volume fractions" model:
   [ 0.0 seconds ]


In [28]:
mit.load_kernels()
mit.load_dictionary(path_commit)
mit.set_threads(1)


-> Resampling LUT for subject "results_script1":
	* Keeping all b0 volume(s)... [ OK ]
	* Normalizing... [ OK ]
   [ 0.1 seconds ]

-> Loading the dictionary:
	* segments from the tracts... [ 102751 fibers and 4894292 segments ]
	* segments from the peaks...  [ 0 segments ]
	* isotropic contributions...  [ 3576 voxels ]
	* post-processing...          [ OK ]
   [ 1.2 seconds ]

-> Distributing workload to different threads:
	* number of threads : 1
	* A operator...  [ OK ]
	* A' operator... [ OK ]
   [ 0.0 seconds ]


In [29]:
mit.build_operator()


-> Building linear operator A:
   [ 3.2 seconds ]


In [30]:
mit.fit(max_iter = 100000000)
## mit.fit(tol_fun = 1e-7, max_iter = 100000000)


-> Fit model

      |     ||Ax-y||     |  Cost function    Abs error      Rel error    |     Abs x          Rel x
------|------------------|-----------------------------------------------|------------------------------
   1  |   2.6691843e+03  |  3.5622725e+06  3.9229763e+07  1.1012567e+01  |  2.2034434e+02  1.4411608e+00
   2  |   1.7444578e+03  |  1.5215665e+06  2.0407060e+06  1.3411876e+00  |  4.5873332e+01  3.9504525e-01
   3  |   1.1488816e+03  |  6.5996448e+05  8.6160198e+05  1.3055278e+00  |  3.3530838e+01  3.7358886e-01
   4  |   7.7451728e+02  |  2.9993851e+05  3.6002597e+05  1.2003326e+00  |  2.4267753e+01  3.4128717e-01
   5  |   5.3683831e+02  |  1.4409769e+05  1.5584082e+05  1.0814943e+00  |  1.7713709e+01  3.0619370e-01
   6  |   3.8543309e+02  |  7.4279332e+04  6.9818354e+04  9.3994321e-01  |  1.3063455e+01  2.7059048e-01
   7  |   2.8747465e+02  |  4.1320837e+04  3.2958496e+04  7.9762411e-01  |  9.7789864e+00  2.3749758e-01
   8  |   2.2153986e+02  |  2.4539955e+04  1.

  77  |   1.3184167e+01  |  8.6911134e+01  2.8182925e-01  3.2427289e-03  |  8.4600536e-02  1.9364770e-02
  78  |   1.3163786e+01  |  8.6642631e+01  2.6850268e-01  3.0989673e-03  |  8.2991238e-02  1.8929092e-02
  79  |   1.3144080e+01  |  8.6383413e+01  2.5921800e-01  3.0007845e-03  |  8.1773951e-02  1.8580754e-02
  80  |   1.3124977e+01  |  8.6132504e+01  2.5090876e-01  2.9130555e-03  |  8.0820605e-02  1.8290240e-02
  81  |   1.3106292e+01  |  8.5887449e+01  2.4505522e-01  2.8532134e-03  |  7.9914883e-02  1.8008491e-02
  82  |   1.3088070e+01  |  8.5648782e+01  2.3866705e-01  2.7865784e-03  |  7.8906606e-02  1.7702020e-02
  83  |   1.3070542e+01  |  8.5419537e+01  2.2924524e-01  2.6837565e-03  |  7.7421409e-02  1.7287902e-02
  84  |   1.3053594e+01  |  8.5198159e+01  2.2137753e-01  2.5983839e-03  |  7.6368704e-02  1.6970203e-02
  85  |   1.3037232e+01  |  8.4984710e+01  2.1344899e-01  2.5116164e-03  |  7.5228845e-02  1.6632996e-02
  86  |   1.3022042e+01  |  8.4786789e+01  1.9792122e-0

In [31]:
mit.save_results(save_coeff=True)


-> Saving results to "Results_VolumeFractions/*":
	* configuration and results:
		- pickle...  [ OK ]
		- txt...  [ OK ]
	* fitting errors:
		- RMSE...  [ 0.071 +/- 0.201 ]
		- NRMSE... [ 0.018 +/- 0.110 ]
	* voxelwise contributions:
		- intra-axonal [ OK ]
		- extra-axonal [ OK ]
		- isotropic    [ OK ]
   [ 0.8 seconds ]


In [56]:
trk_file = os.path.join(path_commit,'dictionary_TRK_fibers.trk')
fib_tmp,trk_hdr_tmp=nib.trackvis.read(trk_file)

In [57]:
len(fib_tmp)

102751

## generate tractography after applying COMMIT with weight > 0, and get fib_after,hdr_after

In [58]:
f = open('/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT/Results_VolumeFractions/xic.txt')


In [59]:
import heapq
data = f.read().splitlines()
data = map(eval,data)
max_index_list = map(data.index, heapq.nlargest(len(data)-data.count(0),data))

#j=0
#for items in range(len(fib_tmp)-1,-1,-1):
#    if data[j] == 0:
#        del fib_tmp[items]
#    j=j+1
#nib.trackvis.write('fibers_after.trk',fib_tmp,trk_hdr_tmp)

In [60]:
len(max_index_list)

1992

In [61]:
j=0
for items in range(len(fib_tmp)-1,-1,-1):
    if items not in max_index_list:
        del fib_tmp[items]
nib.trackvis.write('fibers_after.trk',fib_tmp,trk_hdr_tmp)

In [62]:
fib_after,hdr_after=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_after.trk')

In [63]:
len(fib_after)

1992

In [64]:
CMD = 'TractConverter.py -i fibers_after.trk -o fibers_after.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [65]:
CMD='tckmap fibers_after.tck syn_CLARITY_after.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0

## downsample fibers(before/after) to the original one, take mean distance and std as measurement

## downsample the fibers and get tracks_before, tracks_after, tracks_original

In [66]:
from dipy.tracking.metrics import downsample
import random
tracks_after = [downsample(fib_after[i][0],7) for i in range(len(fib_after))]
tracks_original = [downsample(fib_original[i][0],7) for i in range(len(fib_original))]

from dipy.segment.bundles import bundles_distances_mdf
DM_after = bundles_distances_mdf(tracks_after, tracks_original)
minD_after = [min(DM_after[i,:]) for i in range(len(DM_after))]
list = range(len(fib_before))
slice1 = random.sample(list, len(fib_after))
tracks_before1 = [downsample(fib_before[i][0],7) for i in slice1]

In [68]:
## generate a random list to get equal number of fibers from fib before COMMIT
number_of_trials=10
list = range(len(fib_before))
slice1 = np.zeros((number_of_trials,len(fib_after)),dtype=int)
tracks_before1 = np.zeros((number_of_trials,len(fib_after),7,3))
DM_before1 = np.zeros((number_of_trials,len(fib_after),3000))
minD_before1 = np.zeros((number_of_trials,len(fib_after)))
for i in range(number_of_trials):
    slice1[i] = random.sample(list, len(fib_after))
    tracks_before1[i] = [downsample(fib_before[j][0],7) for j in slice1[i]]
    DM_before1[i] = bundles_distances_mdf(tracks_before1[i], tracks_original)
    minD_before1[i] = [min(DM_before1[i,j,:]) for j in range(len(DM_before1[i]))]

In [69]:
meanD_before = np.zeros(number_of_trials)
stdD_before = np.zeros(number_of_trials)
for i in range(number_of_trials):
    meanD_before[i]= np.mean(minD_before1[i])
    stdD_before[i]= np.std(minD_before1[i])


In [70]:
meanD_before_all = np.mean(meanD_before)
stdD_before_all = np.mean(meanD_before)
meanD_after= np.mean(minD_after)
stdD_after= np.std(minD_after)

In [71]:
meanD_after

0.1301351751529997

In [72]:
meanD_before_all

0.18206487235241015

## compare tracks_after and tracks_before with tracks_original with mean distance and STD

## Compare on a voxel level

original resolution -- 0.015mm

In [73]:
for items in range(len(fib_before)-1,-1,-1):
    if items not in slice1:
        del fib_before[items]
    j=j+1
nib.trackvis.write('fibers_before.trk',fib_before,hdr_before)

In [74]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before.trk')

In [75]:
CMD = 'TractConverter.py -i fibers_before.trk -o fibers_before.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [76]:
CMD='tckmap fibers_before.tck syn_CLARITY_random.nii.gz -vox 0.015' 
subprocess.call(CMD, shell = True)

0

In [77]:
data_original = nib.load('syn_CLARITY.nii.gz')
data_before = nib.load('syn_CLARITY_random.nii.gz')
data_after = nib.load('syn_CLARITY_after.nii.gz')
original = data_original.get_fdata()
random = data_before.get_fdata()
after = data_after.get_fdata()

## normalization
sum_original = sum(sum(sum(data_original.get_fdata())))
sum_before = sum(sum(sum(data_before.get_fdata())))
sum_after = sum(sum(sum(data_after.get_fdata())))
volume_original = data_original.get_fdata()/sum_original
volume_before = data_before.get_fdata()/sum_before
volume_after = data_after.get_fdata()/sum_after

In [78]:
abs_error_before = compClarity(volume_before, volume_original)
abs_error_after = compClarity(volume_after, volume_original)

In [80]:
print(abs_error_before)
print(abs_error_after)

1.6043792757741229
1.8117643295187793


## correlation measure to conduct grey level matching

In [81]:
center_original = (original-sum_original/(191*178*39)).flatten()
center_before = (random - sum_before/(191*178*39)).flatten()
center_after = (after - sum_after/(191*178*39)).flatten()

In [82]:
module_original = np.sqrt(np.dot(center_original,center_original))
module_before = np.sqrt(np.dot(center_before,center_before))
module_after = np.sqrt(np.dot(center_after,center_after))

In [83]:
random_coef = np.dot(center_original.flatten(),center_before.flatten())/(module_original*module_before)
COMMIT_coef = np.dot(center_original.flatten(),center_after.flatten())/(module_original*module_after)

In [84]:
print(random_coef)
print(COMMIT_coef)

0.09500000620597361
0.037355773656436775


## change the resolution here

resolution based on mean distance after COMMIT

CMD='tckmap origin0.015.tck syn_CLARITY_COMMIT_res.nii.gz -vox 0.195' 
subprocess.call(CMD, shell = True)

CMD='tckmap fibers_after.tck syn_CLARITY_after_COMMIT_res.nii.gz -vox 0.195'
subprocess.call(CMD, shell = True)

CMD='tckmap fibers_before.tck syn_CLARITY_random_COMMIT_res.nii.gz -vox 0.195'
subprocess.call(CMD, shell = True)

data_original_COMMIT_res = nib.load('syn_CLARITY_COMMIT_res.nii.gz')
data_before_COMMIT_res = nib.load('syn_CLARITY_random_COMMIT_res.nii.gz')
data_after_COMMIT_res = nib.load('syn_CLARITY_after_COMMIT_res.nii.gz')
## normalization
sum_original_COMMIT_res = sum(sum(sum(data_original_COMMIT_res.get_fdata())))
sum_before_COMMIT_res = sum(sum(sum(data_before_COMMIT_res.get_fdata())))
sum_after_COMMIT_res = sum(sum(sum(data_after_COMMIT_res.get_fdata())))
volume_original_COMMIT_res = data_original_COMMIT_res.get_fdata()/sum_original_COMMIT_res
volume_before_COMMIT_res = data_before_COMMIT_res.get_fdata()/sum_before_COMMIT_res
volume_after_COMMIT_res = data_after_COMMIT_res.get_fdata()/sum_after_COMMIT_res

square_error_before_COMMIT_res = compClarity(volume_before_COMMIT_res, volume_original_COMMIT_res)
square_error_after_COMMIT_res = compClarity(volume_after_COMMIT_res, volume_original_COMMIT_res)

## Compare on a voxel level - binary

original resolution -- 0.015mm

In [85]:
def binarize(volume):
    volume_prime = np.zeros([len(volume),len(volume[1]),len(volume[1][1])])
    for x in range(len(volume)):
        for y in range(len(volume[1])):
            for z in range(len(volume[1][1])):
                if volume[x,y,z]!=0:
                    volume_prime[x,y,z]=1
    return volume_prime

In [86]:
Volume_Original = binarize(volume_original)
Volume_Before = binarize(volume_before)
Volume_After = binarize(volume_after)
error_before = compClarity(Volume_Before, Volume_Original)
error_after = compClarity(Volume_After, Volume_Original)

In [87]:
print(error_before)
print(error_after)

461471.0
247769.0



## change the resolution here

COMMIT resolution 0.186

Volume_Original_COMMIT_res = binarize(volume_original_COMMIT_res)
Volume_Before_COMMIT_res = binarize(volume_before_COMMIT_res)
Volume_After_COMMIT_res = binarize(volume_after_COMMIT_res)
error_before_COMMIT_res = compClarity(Volume_Before_COMMIT_res, Volume_Original_COMMIT_res)
error_after_COMMIT_res = compClarity(Volume_After_COMMIT_res, Volume_Original_COMMIT_res)

## save results

In [89]:
cd ..

/home/zhiwei/Desktop


In [90]:
f = open('meanD_RANDOM_COMMIT.txt','a')  
f.write(str(meanD_before)+', ')
f.write(str(meanD_after)+', ')
f.close()

f = open('stdD_RANDOM_COMMIT.txt','a') 
f.write(str(stdD_before)+', ')
f.write(str(stdD_after)+', ')
f.close()

In [91]:
f = open('VOXEL_B_random_COMMIT.txt','a')
f.write(str(error_before)+', ')
f.write(str(error_after)+', ')
f.close()

In [92]:
f = open('VOXEL_NB_random_commit.txt','a')
f.write(str(square_error_before)+', ')
f.write(str(square_error_after)+', ')
f.close()


NameError: name 'square_error_before' is not defined

In [94]:
f = open('coef_random_COMMIT.txt','a')
f.write(str(random_coef)+', ')
f.write(str(COMMIT_coef)+', ')
f.close()

In [95]:
f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res - BINARIZATION:')
f.write('\nerror random: ' + str(error_before_COMMIT_res))
f.write('\nerror COMMIT: ' + str(error_after_COMMIT_res)) 
f.write('\n\n')
f.close()

NameError: name 'error_before_COMMIT_res' is not defined

In [ ]:
f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res- non-BINARIZATION:')
f.write('\nsquare error random: ' + str(square_error_before_COMMIT_res))
f.write('\nsquare error COMMIT: ' + str(square_error_after_COMMIT_res)) 
f.write('\n\n\n\n\n')
f.close()

In [ ]:
os.rename('results_script1','5_PERCENTER_%d'%order)